In [283]:
import numpy as np
import random
import statistics
random.seed(123456)
mean=[0,0,0,0]
matrix=[[1,0.5,0.5,0.5],
        [0.5,1,0.5,0.5],
        [0.5,0.5,1,0.5],
        [0.5,0.5,0.5,1]]
X=np.random.multivariate_normal(mean,matrix,100)

In [284]:
random.seed(123456)
B= np.random.uniform(0,1,4)

sigma=np.random.normal(0, 1, 100)

Y=np.dot(X,B)+sigma

In [285]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X, Y)
coef_est=reg.coef_

In [286]:
### parametric bootstrap ###
from sklearn.utils import resample
coef=np.zeros((1000, 4))
random.seed(123456)
for x in range(1000):
  bootsample = resample(data, replace=True, n_samples=100, random_state=x)
  mod = LinearRegression().fit(bootsample.iloc[:,bootsample.columns != 0],bootsample.iloc[:,0])
  res = mod.coef_
  coef[x,:]=res
  

In [287]:
#### CI from parametric bootstrap
np.column_stack((2*coef_est-np.quantile(coef, 0.975, axis=0),2*coef_est-np.quantile(coef, 0.025, axis=0)))

array([[-0.28452652,  0.1355114 ],
       [-0.19816111,  0.29558221],
       [ 0.11500291,  0.66902523],
       [-0.68116299, -0.23579135]])

In [288]:
##### non-parametric method: using LS estimator ####
from numpy.linalg import pinv
def beta_p(p,X,Y):
 X=pd.DataFrame(X)
 Beta1=np.dot(np.dot(pinv(np.dot(np.transpose(X.iloc[:,X.columns != p]),X.iloc[:,X.columns != p])),np.transpose(X.iloc[:,X.columns != p])),Y)
 res1=np.subtract(Y,np.dot(X.iloc[:,X.columns != p],Beta1))
 Beta2=np.dot(np.dot(pinv(np.dot(np.transpose(X.iloc[:,X.columns != p]),X.iloc[:,X.columns != p])),np.transpose(X.iloc[:,X.columns != p])),X.iloc[:,p])
 res2=np.subtract(X.iloc[:,p],np.dot(X.iloc[:,X.columns != p],Beta2))
 Beta3=np.dot(np.dot((1/np.dot(np.transpose(res2),res2)),np.transpose(res2)),res1)
 return Beta3


In [289]:
beta_est=np.column_stack((beta_p(0,X,Y),beta_p(1,X,Y),beta_p(2,X,Y),beta_p(3,X,Y)))


In [290]:
conf=np.zeros((1000, 4))
random.seed(123456)
for x in range(1000):
  bootsample = resample(data, replace=True, n_samples=100, random_state=x)
  Y=bootsample[[0]]
  X=bootsample.iloc[:,bootsample.columns != 0]
  X=np.asmatrix(X)
  beta1=beta_p(0,X,Y)
  beta2=beta_p(1,X,Y)
  beta3=beta_p(2,X,Y)
  beta4=beta_p(3,X,Y)
  conf[x,:]=np.column_stack((beta1,beta2,beta3,beta4))
  


In [291]:
#### CI from nonparametric bootstrap
[2*beta_est-np.quantile(conf, 0.975, axis=0),2*beta_est-np.quantile(conf, 0.025, axis=0)]


[array([[-0.26191367, -0.18028342,  0.09560447, -0.68321808]]),
 array([[ 0.16751703,  0.29541276,  0.63753072, -0.22899931]])]